In [1]:
from torch.optim.lr_scheduler import ReduceLROnPlateau, CyclicLR
from load_utils import prepare_data
from train_utils import *
from seq2seq_model import *
import numpy as np
import torch
from tqdm.notebook import tnrange, tqdm_notebook
from Calculate_BLEU import *
import datetime

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = "cpu"

In [3]:
q, a, pairs, vector = prepare_data('train', 'glove.42B.300d/glove.42B.300d.txt', small=True)

Reading train -------
Read 45544 sentence pairs
Counting words
Counted words:
In questions: 14537 words
In answers: 14636 words


In [4]:
# Add EOS at end of each target sentence
pairs = [[line[0], line[1]+' EOS'] for line in pairs ]

In [5]:
matrix_len = q.n_words
weights_matrix = np.zeros((matrix_len, 300))
word_found = 0
for i, word in enumerate(q.word2index):
    try:
        weights_matrix[i] = vector[word]
    except:
        continue

In [6]:
criterion = nn.NLLLoss()
optimizer = 'Adam'
batch_size = 32
lr = 0.001

In [7]:
model = Seq2Seq(batch_size, q.n_words, a.n_words, 300, 100, weights_matrix, 0.2, device, criterion)

RuntimeError: Error(s) in loading state_dict for Embedding:
	size mismatch for weight: copying a param with shape torch.Size([5000, 300]) from checkpoint, the shape in current model is torch.Size([14537, 300]).

In [ ]:
if optimizer == 'Adam':
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, eps=1e-6)
else:
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)

In [ ]:
#scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5, threshold=0.001)
#scheduler = CyclicLR(optimizer, base_lr=0.001, max_lr=0.1, step_size_up=500)

In [ ]:
a.n_words

In [ ]:
from jupyterplot import ProgressPlot

pp = ProgressPlot(x_label='Epoch', plot_names=['train_loss', 'train_accuracy'])

for epoch in tnrange(1000, desc="Total epochs: "):

    print(f'Epoch {epoch + 1}: {datetime.datetime.now()}')

    # Calculte loss
    loss = epoch_train(model, optimizer, batch_size, pairs, q, a, device)
    
    #current_lr = optimizer.param_groups[0]['lr']
    
    print(f'Loss: {loss}')
    #print(f'LR: {current_lr}')
    
    #scheduler.step(loss)
    #scheduler.step()

    # Calculate accuracy
    accuracy = epoch_accuray(model, batch_size, pairs, q, a, device)

    print(f'Train accuracy: {accuracy}')

    # Calculate BLEU Score
    #BLEU_model = CalculateBleu(model, batch_size, pairs, q, a, device)
    #bleu_score = BLEU_model.score()

    #print(f'BLUE score: {bleu_score}')

    # Try to do interactive plot
    pp.update([[loss], [accuracy]])

pp.finalize()
print(f"Optimization ended successfully")
